In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from fastai import *        # Quick accesss to most common functionality
from fastai.text import *   # Quick accesss to NLP functionality
import html

In [3]:
tok_path=Path('data/wikitext-2')

In [4]:
bs=2 
bptt=8

In [5]:
data_tok = TextLMDataBunch.load(tok_path, bs=bs, bptt=bptt)

In [30]:
for i,(x,y) in enumerate(data_tok.train_dl):
    print(x.shape, x.max(), x.float().mean(), x.type())
    print(y.shape, y.max(), y.float().mean(), y.type())
    if i > 6: break

torch.Size([2, 15]) tensor(31567, device='cuda:0') tensor(2147.6335, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 15]) tensor(31567, device='cuda:0') tensor(2264.3667, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 5]) tensor(6856, device='cuda:0') tensor(1088.2001, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 5]) tensor(6856, device='cuda:0') tensor(1498.4000, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 5]) tensor(7970, device='cuda:0') tensor(2224.1001, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 5]) tensor(7970, device='cuda:0') tensor(1489.5000, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 5]) tensor(31564, device='cuda:0') tensor(5171., device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 5]) tensor(31564, device='cuda:0') tensor(5307.8999, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 5]) tensor(20278, device='cuda:0') tensor(2972.3000, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 5]) tensor(20278, device='cu

In [6]:
raw_path=Path('data/wikitext-2-raw')

In [7]:
data_raw = TextLMDataBunch.load(raw_path, bs=bs, bptt=bptt)

In [29]:
for i,(x,y) in enumerate(data_raw.train_dl):
    print(x.shape, x.max(), x.float().mean(), x.type())
    print(y.shape, y.max(), y.float().mean(), y.type())
    if i > 6: break

torch.Size([2, 10]) tensor(4701, device='cuda:0') tensor(615.1000, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 10]) tensor(4701, device='cuda:0') tensor(617.0500, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 8]) tensor(6514, device='cuda:0') tensor(999.0625, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 8]) tensor(6514, device='cuda:0') tensor(998.5000, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 11]) tensor(2897, device='cuda:0') tensor(373.0455, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 11]) tensor(5864, device='cuda:0') tensor(641.6364, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 7]) tensor(5864, device='cuda:0') tensor(691.2858, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 7]) tensor(1282, device='cuda:0') tensor(365.7857, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 5]) tensor(7505, device='cuda:0') tensor(1183.8000, device='cuda:0') torch.cuda.LongTensor
torch.Size([2, 5]) tensor(7505, device='cuda:0') t

In [16]:
tok_ob = data_tok.one_batch(); tok_ob

(tensor([[4679,  319,   18, 1202, 9091,   18, 6078,   32, 6122,   27,  483,  663,
          1585],
         [  26,    8,  398,   13, 3462,   19,  789,   14,    8, 2313,   48, 4729,
          3077]]),
 tensor([[ 319,   18, 1202, 9091,   18, 6078,   32, 6122,   27,  483,  663, 1585,
            10],
         [   8,  398,   13, 3462,   19,  789,   14,    8, 2313,   48, 4729, 3077,
            99]]))

In [24]:
tok_ob_2 = next(iter(data_tok.train_dl)); tok_ob_2

[tensor([[1543,   24, 4174,  144,  869],
         [  61, 6240,   14,  290,   22]], device='cuda:0'),
 tensor([[  24, 4174,  144,  869,  344],
         [6240,   14,  290,   22,   37]], device='cuda:0')]

In [15]:
raw_ob = data_raw.one_batch(); raw_ob

(tensor([[   2,    4,    8,  735, 2630,  216],
         [ 898, 7295, 8747,   96,   21,   15]]),
 tensor([[   4,    8,  735, 2630,  216, 1715],
         [7295, 8747,   96,   21,   15,  289]]))

In [23]:
raw_ob_2 = next(iter(data_raw.train_dl)); raw_ob_2

[tensor([[   2,    4,    8,  319,   52,  651, 1100],
         [  35,  253,    4, 5239,    9,    4, 7232]], device='cuda:0'),
 tensor([[   4,    8,  319,   52,  651, 1100,  819],
         [ 253,    4, 5239,    9,    4, 7232,    9]], device='cuda:0')]

In [12]:
tok_ob[0].shape, tok_ob[0].type(), tok_ob[1].shape, tok_ob[1].type()

(torch.Size([2, 33]),
 'torch.LongTensor',
 torch.Size([2, 33]),
 'torch.LongTensor')

In [14]:
raw_ob[0].shape, raw_ob[0].type(), raw_ob[1].shape, raw_ob[1].type()

(torch.Size([2, 33]),
 'torch.LongTensor',
 torch.Size([2, 33]),
 'torch.LongTensor')

In [65]:
data_tok.show_batch(rows=2)

idx,text
0,= = Villa Grande = = = =
1,a history of that period starting from Philip


In [66]:
data_raw.show_batch(rows=2)

idx,text
0,xxbos = = xxmaj construction
1,status for the league in


In [41]:
vocab_tok = data_tok.train_ds.vocab; vocab_tok.itos[10:20]

['.', 'of', '<unk>', 'and', 'in', 'to', 'a', '=', '"', 'was']

In [42]:
vocab_raw = data_raw.train_ds.vocab; vocab_raw.itos[10:20]

['.', 'of', 'and', 'in', 'to', 'a', '=', '"', 'was', 'on']

In [48]:
[vocab_tok.itos[i] for i in tok_ob[0][0]]

['Note',
 ':',
 'Flags',
 'indicate',
 'national',
 'team',
 'as',
 'defined',
 'under',
 'FIFA',
 'eligibility',
 'rules',
 '.']

In [45]:
vocab_tok.textify(tok_ob[0][0])

'Note : Flags indicate national team as defined under FIFA eligibility rules .'

In [46]:
vocab_raw.textify(raw_ob[0][0])

'xxbos xxmaj andrew xxmaj johnston'